<a href="https://colab.research.google.com/github/saran2811/Entity-recognition/blob/main/entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import json
import pandas as pd
from spacy import displacy
import warnings
import re
warnings.filterwarnings('ignore')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
! python -m spacy download en_core_web_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-27 17:08:47.383236: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 13.1 MB/s eta 0:00:00


In [ ]:
pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=6af61f741ce0a94ecf8a7432b77d318ff1c9c2a9eb20809d93ee5c1c68c6d77b
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [ ]:
import nltk
from nltk.corpus import brown
import contractions
from nltk.stem import WordNetLemmatizer

# **READING THE JSON FILE**

In [ ]:
with open('/content/tweets.json') as jfile:
  d= json.load(jfile)
twt_df=pd.DataFrame.from_dict(d,orient='index')

In [ ]:
twt_df.head()

,tweet_author,tweet_text
1374140386071961602,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...
1374032432173842437,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...
1373902876553048065,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...
1373656782367813635,Toby Eyre,#acalabrutinib is a valuable option in pts int...
1372941634334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...


In [ ]:
twt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43347 entries, 1374140386071961602 to 550579446537678849
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tweet_author  43347 non-null  object
 1   tweet_text    43347 non-null  object
dtypes: object(2)
memory usage: 1015.9+ KB


In [ ]:
twt_df.reset_index(inplace=True,drop=True)
twt_df.head()

,tweet_author,tweet_text
0,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...
1,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...
2,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...
3,Toby Eyre,#acalabrutinib is a valuable option in pts int...
4,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...


**Stopword creation**

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
len(spacy_stopwords)

326

# **CLEANING TWEETS**

In [ ]:
lemmatizer = WordNetLemmatizer()
def tweet_cleaner(text):
    new_text = re.sub(r"'s\b", " is", text)
    new_text = re.sub("#", "", new_text)
    new_text = re.sub("@[A-Za-z0-9]+", "", new_text)
    new_text = re.sub(r"http\S+", "", new_text)
    new_text = contractions.fix(new_text)    
    new_text = re.sub(r"[^a-zA-Z]", " ", new_text)    
    new_text = new_text.strip()
    #Applying stopword removal
    new_text = [token for token in new_text.split() if token not in spacy_stopwords]
    
    new_text = [token for token in new_text if len(token)>2]
    
    cleaned_text = ''
    for token in new_text:
        cleaned_text = cleaned_text + lemmatizer.lemmatize(token) + ' '
    
    return cleaned_text

In [ ]:
cleaned_tweets =[]
for i in range(len(twt_df['tweet_text'])):
  try:
   clean=tweet_cleaner(twt_df['tweet_text'][i])
   cleaned_tweets.append(clean)
  except IndexError:
     twt_df.drop(i,inplace=True)

In [ ]:
twt_df['cleaned_tweet_w/oSW']=cleaned_tweets
twt_df.head()

,tweet_author,tweet_text,cleaned_tweet_w/oSW
0,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...,Scientists conducted Phase study acalabrutinib...
1,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...,This phase Acalabrutinib Venetoclax trial recr...
2,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...,NICE back AstraZenecas Calquence CLL
3,Toby Eyre,#acalabrutinib is a valuable option in pts int...,acalabrutinib valuable option pt intolerant ib...
4,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...,NICE recommended use acalabrutinib patient tre...


In [ ]:
twt_df['cleaned_tweet_w/oSW'][0]

'Scientists conducted Phase study acalabrutinib patient relapsed refractory CLL ibrutinib intolerant found overall response rate '

# **NAMED ENTITY RECOGNITION**

In [ ]:
NER = spacy.load("en_core_web_sm")
twt_df.reset_index(inplace=True,drop=True)

In [ ]:
ent=[]
for i in range(len(twt_df['cleaned_tweet_w/oSW'])):
  
  text1= NER(twt_df['cleaned_tweet_w/oSW'][i])
  for word in text1.ents:
    ent.append(str(word))
  

In [ ]:
entity=[x for x in ent if len(x)>3]

In [ ]:
entity=list(set(entity))

In [ ]:
entity_df = pd.DataFrame({'entity' : [],'frequency':[]})
entity_df

,entity,frequency


In [ ]:
dic={}
for i in range(len(entity)):
  l=entity[i]
  for j in range(len(twt_df['cleaned_tweet_w/oSW'])):
    if re.search(l,twt_df['cleaned_tweet_w/oSW'][j]):
     if l in dic.keys():
      dic[l]+=1
     else:
      dic[l]=1

In [ ]:
entity_df['entity']=dic.keys()
entity_df['frequency']=dic.values()

In [ ]:
entity_df.head(10)

,entity,frequency
0,Robak Montserrat,1
1,Covid Especialistas,3
2,Basically,2
3,TAP Trialsacclerationprogramme CLL JCO,1
4,LIH PhD,1
5,iPhone,1
6,Non Hodgkins,13
7,CALGB Alliance,1
8,JustIn SCCA,1
9,seen year,2


In [ ]:
entity_df.to_csv('objective1.csv')